In [1]:
import sys
import os
import math
import random
import bisect
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import urllib
import subprocess
import re
import pickle
import tempfile
import itertools
import torch
import spacy
import amrlib
import penman

from typing import List, Tuple
from operator import itemgetter 
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertForSequenceClassification
# import qgrid

In [2]:
BASE_DIR = os.path.abspath(os.getcwd()+'/../..')  # /home/gil/dev/NEBULA2/
os.chdir(os.getcwd()+'/../..')

In [3]:
from nebula_api.nebula_enrichment_api import *
from experts.common.RemoteAPIUtility import RemoteAPIUtility
from nebula_api.vlmapi import VLM_API
from nebula_api.atomic2020.comet_enrichment_api import *
from nebula_api.canonisation_api import CANON_API
from bart.commonsense_classifier.commonsense_api import CS_API
import nebula_api.playground_api as pg_api

# from nlp_tools.light_house_generator import LightHouseGenerator

In [4]:
# with torch.cuda.device(0):
nre = NRE_API()
api = RemoteAPIUtility()
vlm = VLM_API()
# mdmmt = mdmmt_api.MDMMT_API()
# comet = Comet("/app/NEBULA2/nebula_api/atomic2020/comet-atomic_2020_BART")
# with torch.cuda.device(0):
ascore = CANON_API()
stog = amrlib.load_stog_model(model_dir="/app/NEBULA2/models/model_stog",device='cuda:1')
gtos = amrlib.load_gtos_model(model_dir="/app/NEBULA2/models/model_gtos", device='cuda:1')
cs_api = CS_API(device_type='gpu', device_id='cuda:0')

# Download cola model
# cola_model = AutoModelForSequenceClassification.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

Milvus Collection Loaded:  vcomet_mdmmt_embedded_event
Milvus Collection Loaded:  vcomet_mdmmt_embedded_place
Milvus Collection Loaded:  vcomet_mdmmt_embedded_actions
INFO:tensorflow:Restoring parameters from /app/NEBULA2/nebula_api/mdmmt_api/ckpts/vggish_model.ckpt


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.

Available class names: ['clip_vit', 'clip_rn', 'mdmmt_max', 'mdmmt_mean', 'mdmmt_legacy']
Loading the model roberta-large...


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.weight', 'clas

The device your are using: cuda:0
Loaded model successfully.
You can use the API now for testing purposes.


In [5]:
def ret_trained_model(path):

    model = BertForSequenceClassification.from_pretrained(
        path, 
        num_labels = 2, 
        output_attentions = False, # Whether the model returns attentions weights.
        output_hidden_states = False, # Whether the model returns all hidden-states.
        return_dict=False
    )
    model.to(torch.device('cpu'))

    return model

# cola_model = ret_trained_model("/app/NEBULA2/bart/sentence_correctness_classifier/weights-20220215Feb02")
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


In [6]:
# movies = ['Movies/114206952',
# 'Movies/114207205',
# 'Movies/114207398',
# 'Movies/114207499',
# 'Movies/114207361',
# 'Movies/114207740',
# 'Movies/114207908',
# 'Movies/114208744',
# 'Movies/114206724',
# 'Movies/114206548',
# 'Movies/114206264']

# GPT_LH_COLLECTION = 'nebula_gpt_lsmdc_playground_candidates'
GPT_LH_COLLECTION = 'nebula_gpt_lsmdc_experts_playground_candidates'
EXPERTS_COLLECTION = 'nebula_comet2020_lsmdc_scored_v02'

def get_playground_movies():
    return(['Movies/114206816', 'Movies/114206849', 'Movies/114206892', 'Movies/114206952', 'Movies/114206999', 'Movies/114207139', 'Movies/114207205', 'Movies/114207240', 'Movies/114207265', 'Movies/114207324', 'Movies/114207361', 'Movies/114207398', 'Movies/114207441', 'Movies/114207474', 'Movies/114207499', 'Movies/114207550', 'Movies/114207668', 'Movies/114207740', 'Movies/114207781', 'Movies/114207810', 'Movies/114207839', 'Movies/114207908', 'Movies/114207953', 'Movies/114207984', 'Movies/114208064', 'Movies/114208149', 'Movies/114208196', 'Movies/114208338', 'Movies/114208367', 'Movies/114208576', 'Movies/114208637', 'Movies/114208744', 'Movies/114208777', 'Movies/114208820', 'Movies/114206358', 'Movies/114206264', 'Movies/114206337', 'Movies/114206397', 'Movies/114206632', 'Movies/114206597', 'Movies/114206691', 'Movies/114206789', 'Movies/114207184', 'Movies/114206548'])

movies = get_playground_movies()


In [7]:
from IPython.display import Javascript
from IPython.display import HTML, display
import base64


def download_video_file(movie, fname='/tmp/video_file.mp4'):    
    if os.path.exists(fname):
        os.remove(fname)
    query = 'FOR doc IN Movies FILTER doc._id == "{}" RETURN doc'.format(movie)
    cursor = api.db.aql.execute(query)
    url_prefix = "http://ec2-18-159-140-240.eu-central-1.compute.amazonaws.com:7000/"
    url_link = ''
    for doc in cursor:
        url_link = url_prefix+doc['url_path']
        url_link = url_link.replace(".avi", ".mp4")   
        print(url_link)
        urllib.request.urlretrieve(url_link, fname) 
    return fname
    # video = cv2.VideoCapture(self.temp_file)
    # fps = video.get(cv2.CAP_PROP_FPS)
    # return(fps, url_link)



def read_video_segm(abspath, t_beg, t_end):
    cmd = f'ffmpeg -y -ss {t_beg} -i {abspath} -max_muxing_queue_size 9999  -loglevel error -f mp4 -vf scale="(floor(112/ih * iw/2))*2:112"  -c:a copy  -movflags frag_keyframe+empty_moov -t {t_end - t_beg} pipe:1 -nostats -hide_banner -nostdin'
    p = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE)
    assert p.returncode == 0, cmd
    buf = p.stdout
    return buf

video_id_cnt = 0    
class VideoElem:
    def __init__(self, fname, t_start=0, t_end=999):
        with open(fname, 'rb') as f:
            #data = base64.standard_b64encode(f.read())
            buf = read_video_segm(fname, t_start, t_end)
            data = base64.standard_b64encode(buf)
        global video_id_cnt
        video_id_cnt += 1
        self.video_id_cnt = video_id_cnt
        elem = HTML(f"""
            <video id="video_{self.video_id_cnt}" autoplay loop muted>
                <source src="data:video/mp4;base64,{data.decode('ascii')}" type="video/mp4">
            </video>        
        """)
        display(elem)
    
    def hide(self):
        js = f'$("#video_{self.video_id_cnt}").hide()'
        display(Javascript(js))
        
    def show(self):
        js = f'$("#video_{self.video_id_cnt}").show()'
        display(Javascript(js))

    def remove(self):
        js = f'$("#video_{self.video_id_cnt}").remove()'
        display(Javascript(js))
        
def mdmmt_video_encode(start_f, stop_f, path='/tmp/video_file.mp4', freq=24):
        t_start = start_f//freq
        t_end = stop_f//freq
        if t_start == t_end:
            t_start = t_start - 1
        print("Start/stop", t_start, " ", t_end)
        if (t_end - t_start) >= 1:
            vemb = mdmmt.encode_video(
                mdmmt.vggish_model,  # adio modality
                mdmmt.vmz_model,  # video modality
                mdmmt.clip_model,  # image modality
                mdmmt.model_vid,  # aggregator
                path, t_start, t_end)
            return(vemb)
        else:
            print("Stage too short")
            return(None)

In [8]:
def flatten(lst): return [x for l in lst for x in l]

def compute_batch_scores(video_emb: torch.Tensor, texts: List[str], normalize=True, **kwargs) -> List[float]:    
    emb_batch = vlm.encode_text(texts, **kwargs)
    if type(emb_batch) == list:
        emb_batch = torch.stack(emb_batch,axis=0)
    if normalize:
        video_emb = video_emb / video_emb.norm(2)
        # print("normalized video norm: {}".format(video_emb.norm(2)))
        n = (emb_batch * emb_batch).sum(axis=1).sqrt()
        emb_batch = emb_batch / n.unsqueeze(1).expand_as(emb_batch)
        # print("normalized text norms:")
        # for emb in emb_batch:
        #     print(emb.norm(2))                        
    return (video_emb.unsqueeze(0).expand_as(emb_batch)*emb_batch).sum(dim=1).cpu().numpy()


def compute_concat_score(image_emb: torch.Tensor, texts: List[str], join_on=',') -> float:
    combined_text = ""
    for t in [x.strip() for x in texts]:
        if t[-1]=='.':
            t = t[:-1]       
        t+=join_on
        t+=' '
        combined_text+=t
    print("Combined: "+combined_text)
    return torch.matmul(image_emb,mdmmt.encode_text(combined_text.strip()) )       

In [9]:
def transform_concept(c):
    exp = re.compile(r"^([a-zA-z]+)-?(\d*)$")
    r = exp.match(c)
    return r.group(1) if r else c

class ConceptManager:
    def __init__(self):
        pass
    def ground_concept(concept):
        return transform_concept(concept)

In [10]:
class SimilarityManager:
    def __init__(self):
        self.nlp = spacy.load('en_core_web_lg')

    def similarity(self, c1, c2):
        if type(c2) is not list:
            c2 = [c2]   
        a = self.nlp(c1)
        targets = self.nlp(' '.join(c2))
        return [a.similarity(x) for x in targets]


smanager = SimilarityManager()


In [11]:
softmax = lambda x: np.exp(x)/sum(np.exp(x))

class SubsetOptimization:
    def __init__(self, video_emb, experts: List, candidates_strings: List[str]):
        self.stog = amrlib.load_stog_model(model_dir="/app/NEBULA2/models/model_stog")
        self.video_emb = video_emb
        self.initial_temp = 10
        self.final_temp = .05
        self.alpha = 0.01
        self.theta = 0.5
        self.experts = experts
        self.candidates_strings = candidates_strings
        self.candidates_amr_strings = self.stog.parse_sents(self.candidates_strings) 
        self.candidates = self.candidates_amr_strings
        self.candidates_amrs = [penman.decode(x) for x in self.candidates_amr_strings]
        self.candidates_similarity = compute_batch_scores(self.video_emb, self.candidates_strings)             
        self.opt_results = []
        self.smanager = SimilarityManager()

        self.coverage_matrix = np.zeros([len(self.experts),len(self.candidates)])
        self.coverage_matrix[:] = np.nan
        for i in range(len(experts)):
            for j in range(len(candidates_strings)):
                self.coverage_matrix[i][j]=self.concept_amr_similarity(self.experts[i],self.candidates_amrs[j])
        self.max_size = int(len(self.experts)*1.5)

    def concept_amr_similarity(self, concept, amr):
        insts = [ConceptManager.ground_concept(x.target) for x in amr.instances()]
        sims = self.smanager.similarity(concept, insts)
        return max(sims)

    def get_coverage(self,i,j):        
        if np.isnan(self.coverage_matrix[i][j]):
            self.coverage_matrix[i][j] = self.concept_amr_similarity(self.experts[i],self.candidates_amrs[j])
        return self.coverage_matrix[i][j]

    def get_expert_coverage(self,state):
        return self.coverage_matrix[:,state].max(axis=1)

    def get_state_coverage(self,state) -> float:
        print("State coverage for {}:".format(state))
        print(self.get_expert_coverage(state))
        return np.mean(self.get_expert_coverage(state))

    # def get_state_coverage(self, state: List[int]) -> float:
    #     experts_coverage = [max([self.get_coverage(i,j) for j in state]) for i in range(len(self.experts))]    # A list of partial coverege        
    #     return sum(experts_coverage) / len(self.experts)

    def get_cost(self, state: List[int]) -> float:
        if not state:
            return 0
        coverage_score = self.get_state_coverage(state)           
        similarity_score = self.candidates_similarity[state].mean().item()
        return -(coverage_score + self.theta*similarity_score)

    # state here is assumed (and guaranteed on return) to be -sorted-
    def get_candidate(self, state: List[int]) -> List[int]:
        def compute_state_arrays(s):
            print("Computing arrays for state: ")
            print(s)
            s_score = self.candidates_similarity[s]
            s_coverage = self.coverage_matrix.mean(axis=0)[s]
            s_max_coverage = self.coverage_matrix.max(axis=0)[s]
            s_fitscore = s_coverage+self.theta*s_score

            return (s_score,s_coverage,s_max_coverage,s_fitscore)

        if not state:
            print("Empty state")
            return [random.randint(0,len(self.candidates_strings)-1)]
            
        rc = state.copy()
        s = np.array(state)
        s_score, s_coverage, s_max_coverage, s_fitscore = compute_state_arrays(s)
               
        if len(state) == self.max_size:
            print("Maximum state size, removing")
            idx = np.argmin(s_fitscore)
            del rc[idx]
            return rc
            
        remove_sentence = random.random()<self.get_state_coverage(state)        
        print("coverage of {} is {}, remove?{}".format(state,self.get_state_coverage(state),remove_sentence))
        if remove_sentence:             # We decide to remove a sentence from the set
            print("Removing")
            probs = softmax(-s_fitscore)
            idx = np.random.multinomial(1,probs).argmax()
            del rc[idx]                   
        else:                           # Add a sentence from the outside
            print("Adding")
            anti_state = []
            for i in range(len(self.candidates_strings)):
                if not i in state:
                    anti_state.append(i)
            s1 = np.array(anti_state)
            s1_score, s1_coverage, s1_max_coverage, s1_fitscore = compute_state_arrays(s1)
            # Pick an expert to try and cover
            probs = softmax(self.get_expert_coverage(s)*10)         # Coverage is in (0,1), so we use low temprature
            expert_to_cover = np.random.multinomial(1,probs).argmax()
            probs = softmax(self.coverage_matrix[expert_to_cover][s1]*10)
            idx_to_add = np.random.multinomial(1,probs).argmax()
            bisect.insort(rc,anti_state[idx_to_add])
            
        return rc



    def get_scored_permutations(self, k):
        n = len(self.candidates)
        return [(x,self.get_cost(list(x))) for x in itertools.permutations(range(n),k)]
        
    def simulated_annealing(self, initial_state):
        self.opt_results = []
        current_temp = self.initial_temp

       # Start by initializing the current state with the initial state
        current_state = initial_state

        while current_temp > self.final_temp:
            next_cand = self.get_candidate(current_state)

            print("current cost: {} ({}). Candidate cost: {} ({})".format(self.get_cost(current_state),current_state,self.get_cost(next_cand),next_cand))

            # Check if next_cand is best so far
            cost_diff = self.get_cost(current_state) - self.get_cost(next_cand)

            # if the new solution is better, accept it
            if cost_diff > 0:
                current_state = next_cand
            # if the new solution is not better, accept it with a probability of e^(-cost/temp)
            else:
                print("chance to move: {}".format(math.exp(cost_diff / current_temp)))
                if random.uniform(0, 1) < math.exp(cost_diff / current_temp):
                    current_state = next_cand
            # decrement the temperature
            current_temp -= self.alpha
            self.opt_results.append(-self.get_cost(current_state))

        return current_state



In [12]:
'''
Get a list of 1-item dictionaries, return a list of the values
'''

def rearrange_concepts(concepts, max_experts=10):
    return concepts[:max_experts]

def permute_sentence(pen, concepts, **kwargs):    
    def replace_instance(g: penman.Graph, changes: List[tuple[int,str]]) -> penman.Graph :
        amr_copy = penman.Graph(triples=g.triples, epidata=g.epidata)
        for (i,val) in changes:
            b = list(amr_copy.triples[i])
            b[2] = val
            amr_copy.triples[i] = tuple(b)
        return amr_copy

    concepts = {k: rearrange_concepts(v, **kwargs) for (k,v) in concepts.items()}    
    insts_list = []
    rc = []
    concept_classes = []
    dims = []
    for i,triplet in enumerate(pen.triples):
        if triplet[1] == ':instance':
            entity_class = ascore.get_class_of_entity(transform_concept(triplet[2]))
            concept_class = concepts[entity_class] if entity_class in concepts.keys() else []
            if triplet[2] not in concept_class:
                concept_class.append(triplet[2])
            insts_list.append((i,triplet, entity_class))
            dims.append(range(len(concept_class)))
            concept_classes.append(concept_class)
    prods = itertools.product(*dims)
    for cand in prods:        
        changes = [(insts_list[i][0],concept_classes[i][d]) for (i,d) in enumerate(cand)]
        rc.append(replace_instance(pen,changes))
    
    return rc    

In [13]:
def rename_variables(pen, prefix='n_'):
    new_triples = []
    for t in pen.triples:
        triple = list(t)
        if triple[0] in pen.variables():
            triple[0] = prefix+triple[0]
        if triple[2] in pen.variables():
            triple[2] = prefix+triple[2]
        new_triples.append(tuple(triple))
    amr_copy = penman.Graph(triples=new_triples)
    return amr_copy            
    

def combine_place(base_amr, place_amr):
    place_copy = rename_variables(place_amr)
    combined_triples = base_amr.triples + place_copy.triples 
    combined_triples.append(tuple([base_amr.top, ':location', place_copy.top]))
    return penman.Graph(triples=combined_triples)


In [14]:
def get_gpt_lh(mid, elem):
    rc = nre.get_scene_from_collection(mid,elem,GPT_LH_COLLECTION)
    return rc['combined']
def get_experts(mid, elem):
    rc = nre.get_scene_from_collection(mid,elem,EXPERTS_COLLECTION)
    return rc['experts'] if rc else None

In [15]:
movies = []
for mid in get_playground_movies():
    movie_info = api.get_movie_info(mid)
    fps = movie_info['fps']
    for i,elem_range in enumerate(movie_info['scene_elements']):
        if elem_range[1]-elem_range[0]+1 < fps:
            continue
        if get_experts(mid,i):
            movies.append((mid,i))

In [16]:
movies

[('Movies/114206952', 1),
 ('Movies/114207324', 0),
 ('Movies/114207908', 0),
 ('Movies/114208149', 2),
 ('Movies/114208196', 1),
 ('Movies/114208338', 0),
 ('Movies/114208576', 0),
 ('Movies/114208744', 0),
 ('Movies/114208744', 2),
 ('Movies/114206337', 0),
 ('Movies/114206337', 1),
 ('Movies/114206548', 0),
 ('Movies/114206548', 1)]

0: ('Movies/114206952', 1)
  Replacemanet:
    ballroom -> bow
  
  Addition:
    "A group of people arrives at the upper level of the ships ballroom" -> "A group of people in suits arrives at the upper level of the ships ballroom where a skipper stands"
      (MDMMT score falls)

1: ('Movies/114207324', 0)
  Replacement:
    man -> nobleman
    
  Addition:

    "A man in a dress bows his head as the gentry pass by in the royal court" -> "A man in a grey dress standing by a window with candles and a vase of flowers bows his head as the gentry pass by in the royal court"
    "A man and a woman facing each other..."
    (MDMMT score falls. These are not attributes)
    (addition of attribute "grey" works)

2: 
  Replacement:
    suit -> helmet, shield
    dress -> ?
    man -> knight, warrior, soldier
    noble -> knight, warrior, soldier

  Addition:
    "A man in a suit and a woman in a dress protect the nobles behind them in the roman arena" -> "A man in a suit and a woman in a dress protect the three nobles behind them in the roman arena"

In [102]:
compute_batch_scores(emb_image,["A man in a helmet and a woman in a dress facing each other protect the nobles behind them in the roman arena", "A man in a suit and a woman in a dress protect the nobles behind them in the roman arena", "A warrior in a helmet and a woman in a dress protect the nobles behind them in the roman arena"])

array([0.3307741 , 0.3080391 , 0.34513435], dtype=float32)

In [129]:
# mid = 'Movies/114208338' # 'Movies/114208338'
mid, elem = movies[2]
movie_info = api.get_movie_info(mid)
emb_image = vlm.encode_video(mid,elem,class_name='mdmmt_mean')
data = nre.get_groundings_from_db(mid,elem)


http://ec2-18-159-140-240.eu-central-1.compute.amazonaws.com:7000/static/development/0025_THE_LORD_OF_THE_RINGS_THE_RETURN_OF_THE_KING_02_55_46_515-02_55_51_946.mp4
Movie info: {'arango_id': 'Movies/114207908', 'description': '0025_THE_LORD_OF_THE_RINGS_THE_RETURN_OF_THE_KING_02_55_46_515-02_55_51_946', 'fps': 23, 'width': 1920, 'height': 1080, 'last frame': 300, 'movie_id': 'f9305a676703419bb958d2c44454293f', 'mdfs': [[2, 57, 112], [116, 122, 129]], 'scene_elements': [[0, 114], [114, 131]]}
fn path: /tmp/video_file.mp4


In [130]:
rc = nre.get_scene_from_collection(mid,elem,GPT_LH_COLLECTION)

In [131]:
rc['combined'][5], rc['source'][5]

(' A man in a suit and a woman in a dress face off with the enemy bravely in a medieval war party',
 ['A man in a suit and a woman in a dress',
  'face off with the enemy bravely',
  'in a medieval war party'])

In [132]:
lh = get_gpt_lh(mid,elem)
experts = get_experts(mid,elem)
scores = compute_batch_scores(emb_image,lh)
cs_scores = cs_api.get_sentences_score(lh)
# First remove all nonsensical sentences (<0.5), then take 5 best by mdmmt
scored_sentences = [x for x in sorted(zip(lh,scores,cs_scores),key=lambda x: -x[2]) if x[2]>0.5]
scored_sentences = sorted(scored_sentences,key=lambda x: -x[1])[:5]
bases = list(list(zip(*scored_sentences))[0])
# bases[0] = "A man in a suit and tie removes the envelope from the owl's mouth in the government capitol"
# bases[1] = 'A man in a suit and tie closes his notes in the government  capitol'

Starting to evaluate the input for the model..


In [133]:
bases, experts, lh

([' A man in a suit and a woman in a dress face off with the enemy bravely in the roman arena',
  ' A man in a suit and a woman provide support to the knights in the roman arena',
  ' A man in a suit and a woman in a dress protect the nobles behind them in the roman arena',
  ' A man in a suit and a woman in a dress face off with the enemy bravely in a medieval war party',
  ' A man in a suit and a woman in a dress protect the nobles behind him on a medieval battlefield'],
 {'person': ['warrior', 'blonde', 'lady', 'soldier', 'knight', 'WHO', 'blond'],
  'artifact': ['helmet', 'shield', 'facing', 'spear', 'weapon', 'watch'],
  'body_part': ['arm', 'shoulder', 'behind'],
  'attribute': ['background', 'three'],
  'location': ['left']},
 [' A man in a suit and a woman in a dress face off with the enemy bravely outside mordor',
  ' A man in a suit and a woman in a dress protect the nobles behind him outside of Mordor',
  ' A man in a suit and a woman in a dress confront his enemies with a v

In [87]:
all_base_amrs = [penman.decode(x) for x in stog.parse_sents(bases)]

In [ ]:
rc = []
for base_amr in all_base_amrs:
    rc.extend(permute_sentence(base_amr,experts, max_experts=2))

In [ ]:
len(rc)

In [ ]:
i = 0
all_cs_scores = []
all_scores = []
all_sentences = []
while i < len(rc):
    print("i is {}".format(i))
    sentences = gtos.generate([penman.encode(x) for x in rc[i:i+3500]])[0]
    # inputs = tokenizer.batch_encode_plus(sentences,padding=True,return_tensors="pt")
    # cola_scores = cola_model(**inputs).logits.softmax(dim=1)[:,1].tolist()
    # cola_scores = cola_model(**inputs)[0].softmax(dim=1)[:,1].tolist()
    cs_scores = cs_api.get_sentences_score(sentences)
    scores = compute_batch_scores(emb_image,sentences,class_name='mdmmt_mean')
    all_sentences.extend(sentences)
    all_cs_scores.extend(cs_scores)
    all_scores.extend(scores)
    i += 3500


In [ ]:
with open('/app/saved_synth_sentences.pickle','wb') as f:
    pickle.dump((all_sentences,all_scores,all_cs_scores),f)

In [ ]:
# z = list(reversed(sorted(zip(all_sentences,all_scores),key=lambda x: x[1])))
z = list(reversed(sorted([x for i,x in enumerate(zip(all_sentences,all_cs_scores,all_scores)) if x[1] > 0.98],key=lambda x: x[1])))

In [ ]:
z1 = list(reversed(sorted(z,key = lambda x:x[2])))

In [ ]:
z1[:20]
# len(z1)

In [ ]:
ind = np.argpartition(all_scores, -10)[-10:]

In [ ]:
[(all_sentences[i], all_scores[i]) for i in ind]

In [ ]:
amr = all_base_amrs[0]

In [ ]:
[amr] = [penman.decode(x) for x in stog.parse_sents(["A man in a suit and tie removes the envelope from the owl's mouth in the government capitol"])]

In [ ]:
t = penman.layout.configure(amr)

In [ ]:
t.node[1][1]

In [ ]:
print(t)